# Model Formulation

## Objective #1

**Indices Definitions:**
- i = the number of census tracts in Allegheny County, and ranges from 1 to 402
- j = the number of potential distribution points (schools utilized), and ranges from 1 to 47

**First-Stage Decision Variable**
- $P$ = the  number of PODs that can be built (given the cost constraints)

**Second-Stage Decision Variables:**
- $X_{j}$ = $\begin{cases}
      1, & \text{if POD j is used}\  \\
      0 , & \text{otherwise}
    \end{cases}$
    
    
- $Y_{ij}$ = $\begin{cases}
      1, & \text{if census tract i is assigned to POD j }\ \\
      0 , & \text{otherwise}
    \end{cases}$
    
**All Other Variables:**
- $h_{i}$ = the total number of households in census tract i 
- $d_{ij}$ = the distance from census tract i to POD j
- $s_{j}$ = the total number of staff available at site j
- $T$ = the total budget
- $c_{p}$ = the cost for each pod, which takes into account the per-POD staffing cost and the per-POD building costs 
- num_pods = the total number of possible pods (47)
- staff_household_ratio = the number of staff members needed per household in need


We will be running a **sensitivity analysis** in addition to satisfying the two objectives below, where we will change the total budget and the cost of each of the PODs. Varying the total budget and the per-POD cost will influence the number of pods that we can build. Through this sensitivity analysis, we will be able to determine the size of the budget that would be needed to build a sufficient number of PODs.


**Objective Function:** 

>  MINIMIZE: $\sum\limits_{i=1}^{402} \sum\limits_{j=1}^{47}  d_{ij} * Y_{ij}   $ 


**Constraints:**

**Cost Contraint for Number of PODs Built**
> $ c_{p} * P  \leq T $


**Number of PODs Assigned Has to Be Less Than Pods Built:**

> $ \sum\limits_{j=1}^{47} X_{j}  \leq P $


**Can't Build More PODs Than Exist**
> $ P  \leq$ num_pods 


**POD Capacity Contstraint:**

If a census tract is assigned to a POD and that POD is actually built, the POD can only assist as many households as staffing capacity allows

> $ \sum\limits_{i=1}^{402} h_{i}  * Y_{ij} \leq$ staff_household_ratio $* s_{j} * X_{j} $ $\forall j $  1,..., 47

**Households Need Access to At Least One POD**

> $ \sum\limits_{j=1}^{47} Y_{ij} \geq 1 $ $\forall i$ 1,...,  402

**Send Households Where PODs Exists:**

> $ Y_{ij} \leq X_{j} $ $\forall i$ 1,..., 402 $\forall j$ 1,..., 40  


**Binary Constraints:**

> $X_{j}$ is binary, $Y_{ij}$ is binary


**Non-Negativity Constraints:**
> $P \geq 0$




## Objective #2


**Additional Decision Variable**

- Z = the maximum access distance
    
    
**All Other Variables:**
- $h_{i}$ = the total number of households in census tract i 
- $P$ = the  number of PODs that can be built (given the cost constraints)
- $d_{ij}$ = the distance from census tract i to POD j
- $s_{j}$ = the total number of staff available at site j
- $T$ = the total budget
- $c_{p}$ = the cost for each pod, which takes into account the per-POD staffing cost and the per-POD building costs 
- num_pods = the total number of possible pods (47)
- staff_household_ratio = the number of staff members needed per household in need


**Objective Function:** 

>  MINIMIZE: Z

**Constraints:**

**Minimizing the Distance & Linking Constraint**

> $ d_{ij} * Y_{ij} \leq Z $

**Cost Contraint for Number of PODs Built**
> $ c_{p} * P  \leq T $


**Number of PODs Assigned Has to Be Less Than Pods Built:**

> $ \sum\limits_{j=1}^{47} X_{j}  \leq P $


**Can't Build More PODs Than Exist**
> $ P  \leq$ num_pods 


**POD Capacity Contstraint:**

If a census tract is assigned to a POD and that POD is actually built, the POD can only assist as many households as staffing capacity allows

> $ \sum\limits_{i=1}^{402} h_{i}  * Y_{ij} \leq$ staff_household_ratio $* s_{j} * X_{j} $ $\forall j $  1,..., 47

**Households Need Access to At Least One POD**

> $ \sum\limits_{j=1}^{47} Y_{ij} \geq 1 $ $\forall i$ 1,...,  402

**Send Households Where PODs Exists:**

> $ Y_{ij} \leq X_{j} $ $\forall i$ 1,..., 402 $\forall j$ 1,..., 40  


**Binary Constraints:**

> $X_{j}$ is binary, $Y_{ij}$ is binary


**Non-Negativity Constraints:**
> $P \geq 0$

## Step 1: Load Data

Load libraries

In [1]:
from gurobipy import *
import pandas as pd
import numpy as np
import csv
import os

Import distances

In [2]:
#get the current working directory
current_directory = os.getcwd()
#go up a level in the directory path to get to the "final-project-dabp" folder
path_parent = os.path.dirname(current_directory)

In [3]:
#dist_path = '..\\data\\02-processed\\distance_matrix.csv'
dist_path = os.path.join(path_parent, "data", "02-processed", "distance_matrix.csv")
dist_txt = np.genfromtxt(dist_path,  dtype=str, delimiter=',', encoding='utf-8-sig')
distance = dist_txt.astype(np.float)

Import POD costs and total budgets - for sensitivity analysis

In [4]:
#pod_costs = pd.read_excel('..\\data\\02-processed\\pod_costs_budget.xlsx', sheet_name = 'pod_costs')[['total_pod_cost']].to_numpy()
#budget = pd.read_excel('..\\data\\02-processed\\pod_costs_budget.xlsx', sheet_name = 'total_budget')[['total_daily_budget']].to_numpy()

pod_costs = pd.read_excel(os.path.join(path_parent, "data", "02-processed", "pod_costs_budget.xlsx"), sheet_name = 'pod_costs')[['total_pod_cost']].to_numpy()
#converts pod_costs to 1-D vector/list (w/o this Gurobi throws an error)
pod_costs = list(map(lambda x: x[0], pod_costs))

budget = pd.read_excel(os.path.join(path_parent, "data", "02-processed", "pod_costs_budget.xlsx"), sheet_name = 'total_budget')[['total_daily_budget']].to_numpy()
#converts budget to 1-D vector/list (w/o this Gurobi throws an error)
budget = list(map(lambda x: x[0], budget))

#upload other limiting variables
household_capacity = pd.read_excel(os.path.join(path_parent, "data", "02-processed", "pod_costs_budget.xlsx"), sheet_name = 'pod_costs')[['hh_capacity']].to_numpy().tolist()
#converts budget to 1-D vector/list (w/o this Gurobi throws an error)
household_capacity = list(map(lambda x: x[0], household_capacity))


Import Household Population Data

In [5]:
#accessing household data
census_data = pd.read_csv(os.path.join(path_parent, "data", "02-processed", "census_geo.csv"))

#household populations with their associated census tract
census_tracts = census_data[["tract","num_hh_tot"]]

#extracts only the household population information
households = list(census_tracts["num_hh_tot"])


Create Indices & Constant Variables

In [6]:
num_census_tracts = distance.shape[0]
num_pods = distance.shape[1] 

census_tracts = range(num_census_tracts)
pods = range(num_pods)

# Step 2: Execute The Model

## Objective #1 Implementation

## Objective #1

**Indices Definitions:**
- i = the number of census tracts in Allegheny County, and ranges from 1 to 402
- j = the number of potential distribution points (schools utilized), and ranges from 1 to 47

**First-Stage Decision Variable**
- $P$ = the  number of PODs that can be built (given the cost constraints)

**Second-Stage Decision Variables:**
- $X_{j}$ = $\begin{cases}
      1, & \text{if POD j is used}\  \\
      0 , & \text{otherwise}
    \end{cases}$
    
    
- $Y_{ij}$ = $\begin{cases}
      1, & \text{if census tract i is assigned to POD j }\ \\
      0 , & \text{otherwise}
    \end{cases}$
    
**All Other Variables:**
- $h_{i}$ = the total number of households in census tract i 
- $d_{ij}$ = the distance from census tract i to POD j
- $s_{j}$ = the total number of staff available at site j
- $T$ = the total budget
- $c_{p}$ = the cost for each pod, which takes into account the per-POD staffing cost and the per-POD building costs 
- num_pods = the total number of possible pods (47)
- staff_household_ratio = the number of staff members needed per household in need


We will be running a **sensitivity analysis** in addition to satisfying the two objectives below, where we will change the total budget and the cost of each of the PODs. Varying the total budget and the per-POD cost will influence the number of pods that we can build. Through this sensitivity analysis, we will be able to determine the size of the budget that would be needed to build a sufficient number of PODs.


**Objective Function:** 

>  MINIMIZE: $\sum\limits_{i=1}^{402} \sum\limits_{j=1}^{47}  d_{ij} * Y_{ij}   $ 


**Constraints:**

**Cost Contraint for Number of PODs Built**
> $ c_{p} * P  \leq T $


**Number of PODs Assigned Has to Be Less Than Pods Built:**

> $ \sum\limits_{j=1}^{47} X_{j}  \leq P $


**Can't Build More PODs Than Exist**
> $ P  \leq$ num_pods 


**POD Capacity Contstraint:**

If a census tract is assigned to a POD and that POD is actually built, the POD can only assist as many households as staffing capacity allows

> $ \sum\limits_{i=1}^{402} h_{i}  * Y_{ij} \leq$ household_capacity * $ X_{j} $ $\forall j $  1,..., 47

**Households Need Access to At Least One POD**

> $ \sum\limits_{j=1}^{47} Y_{ij} \geq 1 $ $\forall i$ 1,...,  402

**Send Households Where PODs Exists:**

> $ Y_{ij} \leq X_{j} $ $\forall i$ 1,..., 402 $\forall j$ 1,..., 40  


**Binary Constraints:**

> $X_{j}$ is binary, $Y_{ij}$ is binary


**Non-Negativity Constraints:**
> $P \geq 0$


Create Decision Variables & Initalize Model

In [7]:
m = Model("minTotal")

#number of pods (first-stage)
p = m.addVar(lb = 0.0, vtype = GRB.INTEGER)

#decision variables (second-stage)
x = m.addVars(pods, vtype = GRB.BINARY)
y = m.addVars(census_tracts, pods, vtype = GRB.BINARY)

Academic license - for non-commercial use only - expires 2022-08-29
Using license file /Users/tobijegede/gurobi.lic


Add Objective Function 

In [8]:
m.setObjective(sum(sum(distance[i,j] * y[i,j] for j in pods) for i in census_tracts))
m.modelSense = GRB.MINIMIZE

Add Constraints

In [ ]:

#in each iteration, we want to capture:
# 1. number of pods constructed
# 2. the budget with for those pods
# 3. the total associated with pod_costs
# 4. the specific pods that are built
# 5. where each census tract gets assigned
# 6. total distance traveled by all households under that budget scenario

total_scenarios = len(budget) * len(pod_costs)

#numpy array to contain the results from the model
pods_with_budget = np.zeros([total_scenarios, 6])


#Note: When I tried to run this with all of the constraints, I got that the setup led to an infeasible solution. 
# I think the issue is with the pod capacity constraint so for now the code below only finds the first-stage decision variable (p)

for iteration in range(total_scenarios):
    for b in range(len(budget)):
        for cost in range(len(pod_costs)):

            total_budget = budget[b]
            cost_per_pod = pod_costs[cost]


            #Number of PODs Assigned Has to Be Less Than Pods Built
            m.addConstr(cost_per_pod * p <= total_budget) 

            #POD Capacity Contstraint
            m.addConstr(sum(x[j] for j in pods) <= p)

            #Can't Build More PODs Than Exist
            m.addConstr(p <= num_pods)

            #You have to build pods
            m.addConstr(p >= 0.0)

            #POD Capacity Constraint -- come back to this constraint
            #note: household_capacity is index by "cost" because len(pod_costs) == len(household_capacity)
#             for j in pods:
#                 m.addConstr(sum(households[i] * y[i,j] for i in census_tracts) <= household_capacity[cost] * x[j])


            #Households Need Access to At Least One POD
            for i in census_tracts:
                m.addConstr(sum(y[i,j] for j in pods) >= 1)

            #Send Households Where PODs Exist
            for i in census_tracts:
                for j in pods:
                    m.addConstr(y[i,j] <= x[j])

            #solve
            m.optimize()

            #capture the results
            total_dist = m.ObjVal
#             for j in pods:
#                 if x[j].x != 0.0:
#                     print(j, x[j].x)
#             total_dist = 0
#             for i in census_tracts:
#                 for j in pods:
#                     if x[j].x == 1.0:
#                         total_dist += distance[i,j]

                        

            pods_with_budget[iteration, 0] = p.x
            pods_with_budget[iteration, 1] = total_budget
            pods_with_budget[iteration, 2] = cost_per_pod
            pods_with_budget[iteration, 5] = total_dist
    #To-do figure out how to store the results that will allow a list type to be one of the stored elements
   # pods_with_budget[iteration, 3] = 
    

print(pods_with_budget)



Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 19300 rows, 18942 columns and 56733 nonzeros
Model fingerprint: 0xff57f383
Variable types: 0 continuous, 18942 integer (18941 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [1e-01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 3 rows and 1 columns
Presolve time: 0.17s
Presolved: 19297 rows, 18941 columns, 56729 nonzeros
Variable types: 0 continuous, 18941 integer (18941 binary)
Found heuristic solution: objective 4276.5900000

Root relaxation: objective 5.858700e+02, 644 iterations, 0.13 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     585.8700000  585.87000  0.00%     -    0s

Explored 0 nodes (644 simplex iter

# Step 3: Visualize & Evaluate Model Results

In [ ]:
# x = 0 
# for b in range(len(budget)):
#     for cost in range(len(pod_costs)):
#         x += 1
#         print(x, budget[b], pod_costs[cost], household_capacity[cost])

#household_capacity[8]